In [ ]:
import arcpy
from src import *

# 1. Indexing the Tiff files

## 1a. choose the folder where the reef DEMs are located

In [ ]:
## enter the folder path to the tif files you are analyzing
src_path = r"C:\Users\ageglio\Documents\NLM_DataRelease\NLM_DataRelease"
# src_path = r"C:\Users\ageglio\OneDrive - DOI\Documents - Reef Mapping\Data Release\NLM_DataRelease\Updated_DataRelease_Surfaces"

## Pattern to filter the bathy tiffs based on their filenames
pattern1 = '.*_BY_0.5m'
pattern2 = None
# pattern2 = '.*0.5m_BY'

# read in the directories bathy files and filter
NLM_files = Utils.list_files(src_path, '.tif')

# Filter based on the provided patterns
if pattern1 and pattern2:
    NLM_files = [i for i in NLM_files if re.search(pattern1, i) or re.search(pattern2, i)]
elif pattern1:
    NLM_files = [i for i in NLM_files if re.search(pattern1, i)]
elif pattern2:
    NLM_files = [i for i in NLM_files if re.search(pattern2, i)]
print("total number of tiffs indexed from filepath", len(NLM_files)) # 18

# custom function to calculate the extents of the tif files
df_tiffs = GetCoordinates().return_min_max_tif_df(NLM_files)
df_tiffs.to_csv("reef_coordinates.csv")
print("filtered tiffs to one per reef totaling: ", len(df_tiffs)) # 18
df_tiffs.head() 

## 1b. Choose a TIF file contained in the dataframe

In [ ]:
## Manual input of the DEM to be used
# input_dem = "dem\dem.tif" #<------------- Large 

## TO INDEX THE the df_tiffs dataframe to the reef you are interested in, change the index id below
input_dem = df_tiffs.filepath[4] #<------------- change index id

# Define the products to be created
products = ["dem"] # just to chunk and merge the dem for formatting reasons
products = ['lbp-3-1', 'lbp-15-2', 'lbp-21-3']
products = ["tpi"]
products = ["slope", "aspect", "roughness", "tpi", "tri", "hillshade"]
products = ["shannon_index"]
# products = ["dem"]
print(input_dem)
print("products to be created", products)

# 2. Derived DEM products

## 2b. Export habitat derivatives

In [ ]:
# Define the output folder path for habitat derivates
out_folder = "habitat_derivatives"
if not os.path.exists(out_folder): os.makedirs(out_folder)

# Get input DEM file and name
dem_name = Utils().sanitize_path_to_name(input_dem)
print("creating habitat derivatives for ", dem_name)
out_dem_folder = os.path.join(out_folder, dem_name)
if not os.path.exists(out_dem_folder): os.mkdir(out_dem_folder)

# Generate output file paths
output_files = {key: os.path.join(out_dem_folder, dem_name +"_"+ key + ".tif") for key in products}

# Process DEM
# Dynamically pass output file paths based on the keys in the output_files dictionary
HabitatDerivatives(chunk_size=None, use_gdal=True).process_dem(
    input_dem,
    shannon_window=21,
    output_slope=output_files.get("slope"),
    output_aspect=output_files.get("aspect"),
    output_roughness=output_files.get("roughness"),
    output_tpi=output_files.get("tpi"),
    output_tri=output_files.get("tri"),
    output_hillshade=output_files.get("hillshade"),
    output_shannon_index=output_files.get("shannon_index"),
    output_lbp_3_1=output_files.get("lbp-3-1"),
    output_lbp_15_2=output_files.get("lbp-15-2"),
    output_lbp_21_3=output_files.get("lbp-21-3"),
    output_dem=output_files.get("dem"),
)

# Perform Infilling
inpainter_ = inpainter(input_dem)
binary_mask = inpainter_.get_filled_data_boundary(
        shrink_pixels=3, min_area=1000, max_iterations=2, smoothing_radius=3)

# Trim Shannon Index
shannon_dem = os.path.join(out_dem_folder, dem_name + "_shannon_index_gdal.tif")
inpainter_.trim_raster(shannon_dem, binary_mask, remove_original=True)

## 2c. Create geomorphons landforms using arcpy 3.4

In [ ]:
# create original landforms from ArcGIS Pro
## Note that once is this the procuct is generated in the local workspace, Arc GIS Pro will not allow you to overwrite the file.
## This is a limitation of the ArcGIS Pro tool, so you will need to delete the file in the local workspace before running this code again.
## Alternatively, you can comment out the line below and just generate the landforms in the next step.
landforms(input_dem).calculate_geomorphon_landforms()

# calculate the 10class solution
output_file10c = landforms(input_dem).classify_bathymorphons(classes="10c")
print(f"Modified raster data saved to {output_file10c}")

# calculate the 6class solution
output_file6c = landforms(input_dem).classify_bathymorphons(classes="6c")
print(f"Modified raster data saved to {output_file6c}")

# calculate the 5class solution
output_file5c = landforms(input_dem).classify_bathymorphons(classes="5c")
print(f"Modified raster data saved to {output_file5c}")

# calculate the 4class solution
output_file4c = landforms(input_dem).classify_bathymorphons(classes="4c")
print(f"Modified raster data saved to {output_file4c}")

In [ ]:
## If you want to save the landforms classification histogram and metadata
landforms.analyze_raster_data(output_file6c)

# 3. Tracklines to shapefile

## 3a. choose a tracklines folder

In [ ]:
from shapely.geometry import LineString, Point, Polygon
import geopandas as gpd

In [ ]:
# Input path to tracklines folders from Qimera
tracklines_folders_paths = r"C:\Users\ageglio\OneDrive - DOI\Documents - Reef Mapping\Tracks_Data_Release_NLMI_Reefs\Tracklines\*\*"
tracklines_folders = glob.glob(tracklines_folders_paths)

# Define the output folder path for shapefiles
out_folder = "shapefiles"
os.makedirs(out_folder, exist_ok=True)

trackline_folder = tracklines_folders[4] #<------------------------ CHANGE INDEX ID
print("folder path chosen: ", trackline_folder)

## 3b. combine and convert tracklines to a shapefile

In [ ]:
def create_tracklines(trackline_folder):
    tracklines_files = glob.glob(os.path.join(trackline_folder, "*.txt"))
    reef_name = os.path.basename(trackline_folder)
    print("creating tracklines shapefile for: ", reef_name)
    output_shapefile_folder = os.path.join(out_folder, reef_name)
    os.makedirs(output_shapefile_folder, exist_ok=True)
    output_shapefile = os.path.join(output_shapefile_folder, f"{reef_name}.shp")

    # Concatenate all dataframes in the list into a single dataframe
    tracklines = pd.concat([pd.read_csv(file, delimiter=',', header=None) for file in tracklines_files], ignore_index=True)
    tracklines.columns = ["UTC", "X", "Y", "Delta"]

    # Get the lat lon coordinates of the raw combined tracklines file
    xy = GetCoordinates.convert_tracklines_to_lat_lon(tracklines, from_wkt="wgs84_16N_OGC_WKT.txt", wgs84_wkt="wgs84_OGC_WKT.txt")

    # Create a GeoDataFrame and save to a shapefile
    gpd.GeoDataFrame(geometry=[LineString(xy)], crs="EPSG:4326").to_file(output_shapefile)

create_tracklines(trackline_folder)
